In [1]:
import numpy as np
import pynq
from dishsoap import CountStreamTest

ol = pynq.Overlay('ex_proj_1.bit', download=True)

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=1, width=80, depth=3)
pp.pprint(ol.ip_dict['axi_dma_0'])

{'addr_range': 65536,
 'bdtype': None,
 'device': <pynq.pl_server.embedded_device.EmbeddedDevice object at 0xaf5bcc88>,
 'driver': <class 'pynq.lib.dma.DMA'>,
 'fullpath': 'axi_dma_0',
 'gpio': {},
 'interrupts': {},
 'mem_id': 'S_AXI_LITE',
 'memtype': 'REGISTER',
 'parameters': {'C_BASEADDR': '0x40400000',
                'C_DLYTMR_RESOLUTION': '125',
                'C_ENABLE_MULTI_CHANNEL': '0',
                'C_FAMILY': 'zynq',
                'C_HIGHADDR': '0x4040FFFF',
                'C_INCLUDE_MM2S': '0',
                'C_INCLUDE_MM2S_DRE': '0',
                'C_INCLUDE_MM2S_SF': '1',
                'C_INCLUDE_S2MM': '1',
                'C_INCLUDE_S2MM_DRE': '0',
                'C_INCLUDE_S2MM_SF': '1',
                'C_INCLUDE_SG': '0',
                'C_INCREASE_THROUGHPUT': '0',
                'C_MICRO_DMA': '0',
                'C_MM2S_BURST_SIZE': '16',
                'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
                'C_M_AXIS_MM2S_TDATA_WIDTH': '32',

                               'fields': {...},
                               'size': 32},
               'S2MM_TAILDESC': {'access': 'read-write',
                                 'address_offset': 64,
                                 'description': 'S2MM DMA Tail Descriptor '
                                                'Pointer Register',
                                 'fields': {...},
                                 'size': 32},
               'S2MM_TAILDESC_MSB': {'access': 'read-write',
                                     'address_offset': 68,
                                     'description': 'S2MM DMA Tail Descriptor '
                                                    'Pointer Register',
                                     'fields': {...},
                                     'size': 32},
               'SG_CTL': {'access': 'read-write',
                          'address_offset': 44,
                          'description': 'Scatter/Gather User and Cache '
        

In [3]:
pp.pprint(ol.ip_dict['count_stream_test_0'])

{'addr_range': 65536,
 'bdtype': None,
 'device': <pynq.pl_server.embedded_device.EmbeddedDevice object at 0xaf5bcc88>,
 'driver': <class 'dishsoap.CountStreamTest'>,
 'fullpath': 'count_stream_test_0',
 'gpio': {},
 'interrupts': {},
 'mem_id': 'S_AXI_registers',
 'memtype': 'REGISTER',
 'parameters': {'COUNTER_WIDTH': '8',
                'C_M_AXIS_output_START_COUNT': '32',
                'C_M_AXIS_output_TDATA_WIDTH': '32',
                'C_S_AXI_registers_ADDR_WIDTH': '4',
                'C_S_AXI_registers_BASEADDR': '0x43C00000',
                'C_S_AXI_registers_DATA_WIDTH': '32',
                'C_S_AXI_registers_HIGHADDR': '0x43C0FFFF',
                'Component_Name': 'zynq7000_preset_count_stream_test_0_0',
                'EDK_IPTYPE': 'PERIPHERAL'},
 'phys_addr': 1136656384,
 'registers': {},
 'state': None,
 'type': 'wbv:user:count_stream_test:0.1'}


In [4]:
from pynq import allocate # for DMA interactions

dma_result = allocate(shape=(4,), dtype=np.uint8)
ol.axi_dma_0.recvchannel.transfer(dma_result)
ol.count_stream_test_0.write(0x0, 1)  # counter_lo
ol.count_stream_test_0.write(0x4, 4) # counter_hi
ol.count_stream_test_0.write(0x8, 2)  # counter start

In [5]:
print('Running?', ol.axi_dma_0.recvchannel.running)
print('Idle?', ol.axi_dma_0.recvchannel.idle)
print('Error?', ol.axi_dma_0.recvchannel.error)
dma_result.invalidate()
print(dma_result)

print(ol.count_stream_test_0.read(0x0))
print(ol.count_stream_test_0.read(0x4))
print(ol.count_stream_test_0.read(0x8))
print(ol.count_stream_test_0.read(0xc))

#ol.axi_dma_0.recvchannel.wait()
#dma_result.invalidate()
#print(dma_result)


Running? True
Idle? False
Error? False
[0 0 0 0]
1
4
2
1


In [6]:
print('Running?', ol.axi_dma_0.recvchannel.running)
print('Idle?', ol.axi_dma_0.recvchannel.idle)
print('Error?', ol.axi_dma_0.recvchannel.error)

Running? True
Idle? False
Error? False


### Notes

I think we need to enable interrupts on the DMA. Other things seem to be working (like the bit clearing in register 0x8)

Might wanna test the other pieces of count_stream_test_0 as well.